# Constraining the total capacity per bus and carrier

In this small example, we limit the nominal capacity of generators of the same production carrier at the same bus. 

Therefore, we introduce a column `nom_min_{carrier}` and `nom_max_{carrier}` in the `buses` dataframe. These are then used as lower and upper bounds of generators of the same carrier at the same bus. 


We start with importing a small example network.

In [1]:
import pandas as pd

import pypsa

In [2]:
n = pypsa.examples.ac_dc_meshed(from_master=True)
n.links_t.p_set = pd.DataFrame(
    index=n.snapshots
)  # remove forced fixed values in optimization

INFO:pypsa.io:Imported network ac-dc-meshed.nc has buses, carriers, generators, global_constraints, lines, links, loads


Now add a second wind generator at bus 'Frankfurt' and limit the combined capacity. 

In [3]:
n.add(
    "Generator",
    "Frankfurt Wind 2",
    bus="Frankfurt",
    capital_cost=120,
    carrier="wind",
    p_nom_extendable=True,
)

n.buses.loc[["Frankfurt", "Manchester"], "nom_min_wind"] = 2000
n.buses.loc[["Frankfurt"], "nom_max_wind"] = 2200

We are running the lopf and check whether the constraint is fulfilled. 

In [4]:
n.optimize()

Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link'], dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC'],
      dtype='object', name='Bus')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link'], dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC'],
      dtype='object', name='Bus')
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:147: UserWarning: coords for

('ok', 'optimal')

In [5]:
n.generators[["p_nom_opt"]]

,p_nom_opt
Generator,
Manchester Wind,2000.000000
Manchester Gas,-0.000000
Norway Wind,895.372552
Norway Gas,91.001539
Frankfurt Wind,100.000000
Frankfurt Gas,884.092697
Frankfurt Wind 2,2100.000000


Looks good! The generators of carrier 'wind' at bus 'Frankfurt' are just the limit of 2200 MW.  